# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop("loan_status", axis = 1)

# Create our target
y = df['loan_status']

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [7]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=78, 
                                                   stratify=y)
X_train.shape
X_train.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
28978,8000.0,0.1180,264.96,MORTGAGE,150000.0,Not Verified,Feb-2019,n,13.02,1.0,...,73.3,75.0,0.0,0.0,324167.0,62427.0,21500.0,39617.0,N,N
39783,20000.0,0.1131,657.72,RENT,60000.0,Not Verified,Jan-2019,n,31.52,0.0,...,87.5,50.0,0.0,0.0,81223.0,54620.0,16000.0,52556.0,N,N
24072,20000.0,0.1797,507.55,RENT,50000.0,Source Verified,Feb-2019,n,27.99,0.0,...,100.0,75.0,0.0,0.0,68281.0,40211.0,25000.0,42281.0,N,N
7540,15000.0,0.1390,511.94,MORTGAGE,72000.0,Source Verified,Mar-2019,n,6.05,1.0,...,90.0,12.5,0.0,0.0,95250.0,8449.0,13600.0,7550.0,N,N
16169,10000.0,0.1797,361.38,RENT,41000.0,Not Verified,Feb-2019,n,31.21,0.0,...,100.0,100.0,0.0,0.0,53860.0,33625.0,15300.0,38560.0,N,N


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
#X_train.drop(['home_ownership', 
#'verification_status', 'issue_d', 'pymnt_plan', 
#'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 'debt_settlement_flag'], axis=1, inplace=True)
#X_test.drop(['home_ownership', 
#'verification_status', 'issue_d', 'pymnt_plan', 
#'initial_list_status', 'next_pymnt_d', 'application_type', 'hardship_flag', 'debt_settlement_flag'], axis=1, inplace=True)

X_train.drop(['debt_settlement_flag'], axis=1, inplace=True)
X_test.drop(['debt_settlement_flag'], axis=1, inplace=True)
X_scaler = scaler.fit(X_train)

In [22]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [23]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [25]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.5

In [26]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [17118,     0]])

In [27]:
# Print the imbalanced classification report
# YOUR CODE HERE
y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.59      0.86      0.04      0.71      0.49        87
   low_risk       1.00      0.86      0.59      0.93      0.71      0.52     17118

avg / total       0.99      0.86      0.59      0.92      0.71      0.52     17205



In [28]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07481735006065673, 'initial_list_status'),
 (0.06238929252337991, 'total_rec_prncp'),
 (0.056008821419548015, 'out_prncp'),
 (0.0535501627498247, 'total_acc'),
 (0.05293520692916732, 'revol_bal'),
 (0.03184639427089025, 'int_rate'),
 (0.01935143591589689, 'open_il_12m'),
 (0.0189969704667074, 'annual_inc'),
 (0.01871066800180638, 'acc_open_past_24mths'),
 (0.01778809193766382, 'delinq_amnt'),
 (0.017482033083366023, 'home_ownership'),
 (0.017409313805030004, 'installment'),
 (0.01725872862142521, 'pub_rec'),
 (0.017195172760931, 'tot_cur_bal'),
 (0.016583360798122313, 'open_acc'),
 (0.016563003706179996, 'max_bal_bc'),
 (0.01633889370500539, 'num_tl_30dpd'),
 (0.015517544707834468, 'total_rev_hi_lim'),
 (0.015329078732479027, 'delinq_2yrs'),
 (0.015100131898968786, 'all_util'),
 (0.015067018700234395, 'num_tl_op_past_12m'),
 (0.015027925590156407, 'open_il_24m'),
 (0.014791383430765423, 'last_pymnt_amnt'),
 (0.014684630623518043, 'inq_last_12m'),
 (0.014063872640507395, 'acc_now_de

### Easy Ensemble Classifier

In [29]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators = 500, random_state = 1)

In [30]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
ee_model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=500, random_state=1)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.